In [1]:
import sys  
import os
import pandas as pd
import numpy as np
import swat

In [2]:
conn = swat.CAS(os.environ['CASHOST'], os.environ['CASPORT'])
conn.loadactionset('deeplearn')
conn.loadactionset('sampling')
conn.loadactionset('textParse')
conn.loadactionset('decisionTree')

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    
import swat.cas.datamsghandlers as dmh

NOTE: Added action set 'deeplearn'.
NOTE: Added action set 'sampling'.
NOTE: Added action set 'textParse'.
NOTE: Added action set 'decisionTree'.


In [3]:
import dlpy
from dlpy import Sequential
from dlpy import *
from dlpy.model import Model, Optimizer, AdamSolver
from dlpy.model import TextParms
from dlpy.blocks import Bidirectional
from dlpy.applications import TextClassification

In [4]:
#path = 'C:/Users/shpath/Documents/korean/'
article_train = conn.upload(path + 'news_training.sas7bdat', casout=dict(name='article_train', replace=True))
train = conn.CASTable(name='article_train', casout='train')
train.tail()

NOTE: Cloud Analytic Services made the uploaded file available as table ARTICLE_TRAIN in caslib CASUSER(shpath).
NOTE: The table ARTICLE_TRAIN has been created in caslib CASUSER(shpath) from binary data uploaded to Cloud Analytic Services.


,docID,title,content,Category
3995,51e91238eda5cbf2,"쿠데타 관련 장성 47명 강퇴 터키, 99명 장군진급 잔치",레제프 타이이프 에르도안 대통령왼쪽이 군사평의회의 대규모 진급 결정을 승인하는 서명...,C5.중동아프리카
3996,394c7a2574ddaceb,"美히스패닉계 2명, 살인혐의로 23년 복역 후 무죄 석방",미국의 히스패닉계 남성 2명이 살인혐의로 수감된 지 23년 만에 무죄 석방됐다. 시...,C5.중동아프리카
3997,3ee3c09e78bf8c03,"터키, 3개월간 에르도안이 황제…사회각계서 6만명 숙청",에르도안 지지자들 [AFP=연합뉴스] 국가비상사태 선포…2002년 남동부서 종료된 ...,C5.중동아프리카
3998,8826aa7529070c78,시리아 정부군에 투항하는 알레포 반군,【베이루트=AP/뉴시스】시리아 정부군이 북부 알레포에서 반군 포위작전을 3주간 펼치...,C5.중동아프리카
3999,1b6d0ec5ce31bc9b,아프로 머리 소중하다 남아공 여학생들 차별 반대시위,남아공 프리토리아 여고 앞에서 생머리 퍼머를 강요하는 학칙에 항의해 시위를 벌이고 ...,C5.중동아프리카


In [5]:
content=train['content'].tolist()

In [6]:
import pandas as pd
embeddings = pd.DataFrame.from_csv(path + 'w2v_300.txt',sep='\t')
conn.upload_frame(embeddings, casout=dict(name='w2v_300', replace=True))

C:\Users\shpath\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


NOTE: Cloud Analytic Services made the uploaded file available as table W2V_300 in caslib CASUSER(shpath).
NOTE: The table W2V_300 has been created in caslib CASUSER(shpath) from binary data uploaded to Cloud Analytic Services.


CASTable('W2V_300', caslib='CASUSER(shpath)')

In [9]:
model = TextClassification(conn, neurons=128, n_blocks=2)


NOTE: Output layer added.
NOTE: Model compiled successfully.


In [14]:
n_epochs=1000

for i in range(10):
    res = conn.sampling.stratified(table='article_train', partind=True,
                            output=dict(
                                casout=dict(name='article_partitioned',
                                            replace=True),
                                copyvars='ALL',
                                partindname='partition'),
                            samppct=10)  # sampling rate for validation set
                            

    train = conn.CASTable(name='article_partitioned', where='partition=0', casout='train')
    valid = conn.CASTable(name='article_partitioned', where='partition=1', casout='valid')

    model.fit(data=train, inputs='content', texts='content', target='category', nominals='category', valid_table=valid,
          text_parms=TextParms(init_input_embeddings='w2v_300'), record_seed=909455678,
          mini_batch_size=4, max_epochs=n_epochs, lr=0.01, log_level=2)
    
    n_epochs=15

NOTE: Using SEED=464828720 for sampling.
NOTE: Training from scratch.
NOTE:  The Synchronous mode is enabled.
NOTE:  The total number of parameters is 626437.
NOTE:  The approximate memory cost is 3196.00 MB.
NOTE:  Loading weights cost       0.00 (s).
NOTE:  Initializing each layer cost      35.49 (s).
NOTE:  The total number of threads on each worker is 96.
NOTE:  The total mini-batch size per thread on each worker is 4.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 384.
NOTE:  Target variable: Category
NOTE:  Number of levels for the target variable:      5
NOTE:  Levels for the target variable:
NOTE:  Level      0: C1.금융            
NOTE:  Level      1: C2.IT모바일       
NOTE:  Level      2: C3.부동산         
NOTE:  Level      3: C4.방송연예      
NOTE:  Level      4: C5.중동아프리카
NOTE:  Number of input variables:     1
NOTE:  Number of text input variables:      1
NOTE:  Epoch Learning Rate        Loss  Fit Error  Validation Loss Validation Error   Time(s)

NOTE:  84         0.01           1.073     0.3947            1.062             0.42     3.89
NOTE:  85         0.01           1.054     0.3837            1.058             0.42     4.06
NOTE:  86         0.01           1.055     0.4029            1.054           0.4175     3.71
NOTE:  87         0.01           1.064      0.412             1.05            0.415     4.14
NOTE:  88         0.01           1.052      0.401            1.046           0.3925     3.74
NOTE:  89         0.01           1.041     0.4067            1.044             0.39     4.04
NOTE:  90         0.01            1.07     0.4115            1.036           0.4125     4.21
NOTE:  91         0.01           1.039     0.3986            1.034           0.4075     4.25
NOTE:  92         0.01           1.082     0.4111            1.029              0.4     3.95
NOTE:  93         0.01           1.028     0.3995            1.029            0.405     4.20
NOTE:  94         0.01           1.041     0.4048            1.021    

NOTE:  173        0.01          0.8262     0.3139           0.8236            0.295     3.89
NOTE:  174        0.01          0.8381     0.3091           0.8162            0.295     3.90
NOTE:  175        0.01          0.8124     0.2957           0.8171             0.29     3.99
NOTE:  176        0.01          0.8282     0.3048           0.8174             0.29     4.25
NOTE:  177        0.01          0.8346     0.3058            0.811           0.2925     4.19
NOTE:  178        0.01            0.84     0.3115           0.8094             0.29     3.86
NOTE:  179        0.01          0.8315     0.3024           0.8045           0.2825     3.76
NOTE:  180        0.01          0.8337     0.3034           0.8053           0.2825     4.18
NOTE:  181        0.01          0.8356     0.3154           0.8021             0.29     3.89
NOTE:  182        0.01          0.8486     0.3029            0.803           0.2825     4.10
NOTE:  183        0.01          0.8028     0.2933           0.7971    

NOTE:  263        0.01           0.699     0.2663           0.6644            0.245     3.98
NOTE:  264        0.01          0.6754     0.2481           0.6626           0.2375     3.74
NOTE:  265        0.01          0.6989     0.2611             0.66           0.2475     4.22
NOTE:  266        0.01            0.69     0.2534           0.6638            0.245     3.74
NOTE:  267        0.01          0.6704     0.2486           0.6578           0.2325     3.96
NOTE:  268        0.01          0.6562     0.2303           0.6673            0.255     3.99
NOTE:  269        0.01          0.6663     0.2481           0.6625            0.245     4.25
NOTE:  270        0.01          0.6568     0.2365           0.6577           0.2425     3.91
NOTE:  271        0.01          0.6647      0.249           0.6521           0.2425     3.78
NOTE:  272        0.01          0.6779     0.2476           0.6536             0.25     4.23
NOTE:  273        0.01          0.6691     0.2481           0.6484    

NOTE:  355        0.01          0.5706     0.2062           0.5737           0.2025     3.90
NOTE:  356        0.01          0.5854     0.2337           0.5709              0.2     4.05
NOTE:  357        0.01          0.5804     0.2154           0.5691            0.195     3.85
NOTE:  358        0.01           0.578      0.213           0.5639            0.185     3.82
NOTE:  359        0.01          0.5624     0.2183           0.5641           0.1975     3.95
NOTE:  360        0.01          0.6009     0.2178            0.593             0.21     4.34
NOTE:  361        0.01          0.5825      0.225           0.5644           0.1925     4.08
NOTE:  362        0.01           0.572     0.2226           0.5577           0.1975     3.92
NOTE:  363        0.01          0.5332     0.1976           0.5627              0.2     4.04
NOTE:  364        0.01          0.5558     0.1938            0.568           0.1975     4.26
NOTE:  365        0.01           0.578     0.2168           0.5906    

...

NOTE:  812        0.01          0.2852    0.09471           0.4553           0.1525     4.00
NOTE:  813        0.01          0.2788     0.1038             0.44           0.1425     3.75
NOTE:  814        0.01          0.3613     0.1293           0.4221           0.1275     3.96
NOTE:  815        0.01          0.2864    0.09856           0.4651             0.15     3.84
NOTE:  816        0.01          0.2876      0.112            0.434            0.145     4.00
NOTE:  817        0.01           0.305     0.1096           0.4258           0.1425     3.82
NOTE:  818        0.01          0.2721    0.09615           0.4287             0.14     3.96
NOTE:  819        0.01          0.2722    0.09904           0.4155             0.13     4.06
NOTE:  820        0.01          0.2905     0.1154           0.4318           0.1425     3.75
NOTE:  821        0.01          0.2981    0.09904            0.425             0.14     4.13
NOTE:  822        0.01          0.2823     0.1053           0.4444    

NOTE:  903        0.01          0.2607    0.08558           0.4365             0.14     3.99
NOTE:  904        0.01           0.273     0.1019           0.4433             0.14     3.97
NOTE:  905        0.01          0.2914     0.1072           0.4182             0.15     3.75
NOTE:  906        0.01           0.254    0.09615           0.4518           0.1325     4.14
NOTE:  907        0.01          0.2553    0.09567            0.425             0.13     4.28
NOTE:  908        0.01          0.2577    0.09712           0.4302           0.1425     4.15
NOTE:  909        0.01          0.2613    0.09183           0.4837           0.1475     4.08
NOTE:  910        0.01          0.2921     0.1082           0.4487           0.1425     4.01
NOTE:  911        0.01          0.2483     0.0875           0.4199           0.1275     4.06
NOTE:  912        0.01          0.2479    0.08269           0.4332           0.1425     4.00
NOTE:  913        0.01          0.2693    0.09712           0.4358    

NOTE:  993        0.01          0.2043    0.07212           0.4256           0.1325     4.00
NOTE:  994        0.01          0.2144    0.07692           0.4598           0.1425     3.70
NOTE:  995        0.01          0.2039    0.07404           0.4341            0.135     3.79
NOTE:  996        0.01          0.2242    0.07356           0.4199            0.145     4.07
NOTE:  997        0.01          0.2556    0.09423           0.4087           0.1325     4.13
NOTE:  998        0.01          0.2353     0.0899           0.4259             0.14     4.30
NOTE:  999        0.01          0.2181    0.07644           0.4483           0.1425     3.74
NOTE:  The optimization reached the maximum number of epochs.
NOTE:  The total time is    3983.26 (s).
NOTE: Using SEED=16098325 for sampling.
NOTE: Training based on existing weights.
NOTE:  The Synchronous mode is enabled.
NOTE:  The total number of parameters is 626437.
NOTE:  The approximate memory cost is 3196.00 MB.
NOTE:  Loading weights co

NOTE: Using SEED=474003764 for sampling.
NOTE: Training based on existing weights.
NOTE:  The Synchronous mode is enabled.
NOTE:  The total number of parameters is 626437.
NOTE:  The approximate memory cost is 3195.00 MB.
NOTE:  Loading weights cost       0.02 (s).
NOTE:  Initializing each layer cost      33.92 (s).
NOTE:  The total number of threads on each worker is 96.
NOTE:  The total mini-batch size per thread on each worker is 4.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 384.
NOTE:  Target variable: Category
NOTE:  Number of levels for the target variable:      5
NOTE:  Levels for the target variable:
NOTE:  Level      0: C1.금융            
NOTE:  Level      1: C2.IT모바일       
NOTE:  Level      2: C3.부동산         
NOTE:  Level      3: C4.방송연예      
NOTE:  Level      4: C5.중동아프리카
NOTE:  Number of input variables:     1
NOTE:  Number of text input variables:      1
NOTE:  Epoch Learning Rate        Loss  Fit Error  Validation Loss Validation Er

NOTE: Training based on existing weights.
NOTE:  The Synchronous mode is enabled.
NOTE:  The total number of parameters is 626437.
NOTE:  The approximate memory cost is 3196.00 MB.
NOTE:  Loading weights cost       0.02 (s).
NOTE:  Initializing each layer cost      34.14 (s).
NOTE:  The total number of threads on each worker is 96.
NOTE:  The total mini-batch size per thread on each worker is 4.
NOTE:  The maximum mini-batch size across all workers for the synchronous mode is 384.
NOTE:  Target variable: Category
NOTE:  Number of levels for the target variable:      5
NOTE:  Levels for the target variable:
NOTE:  Level      0: C1.금융            
NOTE:  Level      1: C2.IT모바일       
NOTE:  Level      2: C3.부동산         
NOTE:  Level      3: C4.방송연예      
NOTE:  Level      4: C5.중동아프리카
NOTE:  Number of input variables:     1
NOTE:  Number of text input variables:      1
NOTE:  Epoch Learning Rate        Loss  Fit Error  Validation Loss Validation Error   Time(s)
NOTE:  0          0.01     

In [ ]:
model.save_to_table('/bigdisk/lax/shpath/korean/')

In [10]:
model.load_weights('/bigdisk/lax/shpath/korean/text_classifier_weights.sashdat')

NOTE: Model weights attached successfully!
NOTE: text_classifier_weights_attr.sashdat is used as weigths attribute.
NOTE: Model attributes attached successfully!


In [11]:
article_test = conn.upload(path + 'news_article_0.sas7bdat', casout=dict(name='article_test', replace=True))
test = conn.CASTable(name='article_test', casout='test')
test.tail()
test_df = pd.DataFrame(conn.CASTable('article_test').to_frame())
test_df.rename(columns={'category_answer':'category'}, inplace=True)
conn.upload(test_df, casout=dict(name='article_test', replace=True))



NOTE: Cloud Analytic Services made the uploaded file available as table ARTICLE_TEST in caslib CASUSER(shpath).
NOTE: The table ARTICLE_TEST has been created in caslib CASUSER(shpath) from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table ARTICLE_TEST in caslib CASUSER(shpath).
NOTE: The table ARTICLE_TEST has been created in caslib CASUSER(shpath) from binary data uploaded to Cloud Analytic Services.


[caslib]

 'CASUSER(shpath)'

[tableName]

 'ARTICLE_TEST'

[casTable]

 CASTable('ARTICLE_TEST', caslib='CASUSER(shpath)')

+ Elapsed: 0.593s, user: 0.365s, sys: 0.208s, mem: 200mb

In [12]:
res = model.evaluate(data='article_test',top_probs=10, text_parms=TextParms(init_input_embeddings='w2v_300'))
res

[ScoreInfo]

                                Descr         Value
 0        Number of Observations Read          1000
 1        Number of Observations Used          1000
 2        Misclassification Error (%)          14.5
 3  Top 5 Misclassification Error (%)             0
 4                         Loss Error      0.451243

[OutputCasTables]

             casLib              Name  Rows  Columns  \
 0  CASUSER(shpath)  Valid_Res_2ulCmk  1000       24   
 
                                             casTable  
 0  CASTable('Valid_Res_2ulCmk', caslib='CASUSER(s...  

+ Elapsed: 38.4s, user: 46.7s, sys: 13.2s, mem: 5.77e+03mb

In [13]:
model.valid_conf_mat

,category,Col1,Col2,Col3,Col4,Col5
0,C1.금융,158.0,10.0,18.0,6.0,8.0
1,C2.IT모바일,9.0,178.0,6.0,1.0,6.0
2,C3.부동산,42.0,4.0,148.0,3.0,3.0
3,C4.방송연예,9.0,4.0,1.0,185.0,1.0
4,C5.중동아프리카,3.0,3.0,6.0,2.0,186.0
